In [ ]:
# Dowload the data
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Load the data
houses = pd.read_csv('/kaggle/input/paris-housing-price-prediction/ParisHousing.csv')
houses.head()

In [ ]:
houses.info()

In [ ]:
houses.describe()

In [ ]:
# Looking for missing values
houses.isnull().any(axis = 'rows')

In [ ]:
houses.hist(bins = 50, figsize = (20,20))
plt.show()

In [ ]:
# Strong correlation of squared meters and price
import seaborn as sns
corr = houses.corr()
plt.figure(figsize = (12, 8))
sns.heatmap(corr, annot = True)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(houses['squareMeters'], houses['price'])

In [ ]:
from sklearn.model_selection import train_test_split
houses = houses.drop('cityCode', axis = 1)
train_set, test_set = train_test_split(houses, test_size = 0.2, random_state = 42)
len(train_set), len(test_set)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

pipeline = ColumnTransformer([('encoder', OneHotEncoder(), ['made']),
                              ('scaler', StandardScaler(), ['squareMeters', 'numberOfRooms', 'floors', 'cityPartRange',
                                                            'numPrevOwners','basement','attic', 'garage', 'hasGuestRoom'])])

In [ ]:
# Preparing data
train_set_predictors = train_set.drop('price', axis = 1)
train_prepared = pipeline.fit_transform(train_set_predictors) 
print(len(houses['made'].value_counts())) # Extra columns One Hot Encoder of 'made'
train_labels = train_set['price'].copy()
train_prepared.toarray()

In [ ]:
# Models
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

lin_reg = LinearRegression()
sgd_reg = SGDRegressor()
forest_reg = RandomForestRegressor()
svr_reg = SVR()

models = ['lin_reg', 'sgd_reg', 'forest_reg', 'svr_reg']
dic_models = {}
means = []

plt.figure(figsize = (12,6))
for model in models:
    scores = cross_val_score(eval(model), train_prepared, train_labels, cv = 3, scoring = 'neg_mean_squared_error')
    scores_ = np.sqrt(-scores)
    dic_models[model] = scores
    dic_models[f'{model}_mean'] = scores_.mean()
    means.append(scores_.mean())
    
plt.bar(models, means)
plt.title('mean RMSE')
plt.show()
dic_models

In [ ]:
# As we see before the correlation with the price is 1, so linear regression is just do perfect predictions,let's see
test_set_predictors = test_set.drop('price', axis = 1)
test_set_prepared = pipeline.transform(test_set_predictors)
test_labels = test_set['price'].copy()
lin_reg.fit(train_prepared, train_labels)
pred = lin_reg.predict(test_set_prepared)
score = mean_squared_error(test_labels, pred)
score_ = np.sqrt(score)
score_

In [ ]:
from sklearn.metrics import r2_score # Perfect r2 score
r2_score(test_labels, test_labels)